In [1]:
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [2]:
try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

# env_name = "CartPole-v1"
# env_name = "MountainCar-v0"
# env_name = "MountainCarContinuous-v0"
# env_name = "Acrobot-v1"
# env_name = "Pendulum-v0"
env_name = "FrozenLake-v0"
env_name = "FrozenLakeNoSlip-v0"
env = gym.make(env_name)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
type(env.action_space)

Observation space: Discrete(16)
Action space: Discrete(4)


gym.spaces.discrete.Discrete

In [3]:
class Agent():
    def __init__(self, env):
        self.is_discrete = \
            type(env.action_space) == gym.spaces.discrete.Discrete
        
        if self.is_discrete:
            self.action_size = env.action_space.n
            print("Action size:", self.action_size)
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
            print("Action range:", self.action_low, self.action_high)
        
    def get_action(self, state):
        if self.is_discrete:
            action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low,
                                       self.action_high,
                                       self.action_shape)
        return action

In [4]:
class QAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        print("State size:", self.state_size)
        
        self.eps = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.build_model()
        
    def build_model(self):
        self.q_table = 1e-4*np.random.random([self.state_size, self.action_size])
        
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.eps else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate * np.max(q_next)
        
        q_update = q_target - self.q_table[state,action]
        self.q_table[state,action] += self.learning_rate * q_update
        
        if done:
            self.eps = self.eps * 0.99
        
agent = QAgent(env)

Action size: 4
State size: 16


In [5]:
total_reward = 0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state,action,next_state,reward,done))
        state = next_state
        total_reward += reward
        
        print("s:", state, "a:", action)
        print("Episode: {}, Total reward: {}, eps: {}".format(ep,total_reward,agent.eps))
        env.render()
        print(agent.q_table)
        time.sleep(0.05)
        clear_output(wait=True)

s: 5 a: 1
Episode: 99, Total reward: 11.0, eps: 0.36603234127322926
  (Down)
SFFF
FHFH
FFFH
HFFG
[[6.56116217e-05 4.24464423e-05 7.61640411e-05 8.92844908e-05]
 [3.71512389e-05 5.92853324e-05 8.76648413e-05 9.01200970e-05]
 [7.09607170e-05 1.01753661e-04 5.95713498e-05 2.93910558e-05]
 [2.78058890e-05 7.15554972e-05 3.65918475e-05 1.69015040e-05]
 [6.30075691e-05 5.17929449e-05 5.33316337e-05 7.04731245e-05]
 [6.39761516e-05 9.84640666e-05 9.60240306e-05 1.51792894e-05]
 [1.58257106e-06 2.93845813e-04 6.49157762e-05 5.95094371e-05]
 [2.69191630e-05 5.41587356e-05 3.55939376e-05 7.10073882e-05]
 [9.70107400e-05 8.54954470e-05 4.63052493e-06 4.15226726e-05]
 [5.38303821e-06 8.13909845e-05 7.88113482e-05 4.86611207e-05]
 [6.08782647e-05 6.15019862e-03 6.74209614e-05 7.58401888e-05]
 [9.08960630e-05 7.56294749e-05 1.98927348e-05 6.10832572e-05]
 [8.11245014e-05 2.41497402e-05 6.33098004e-05 8.75079460e-05]
 [6.48720497e-05 5.53993859e-05 5.38129854e-05 9.47141786e-05]
 [7.01938959e-05 2.54